## Семинар 9: Тема: Классификация текстов.

Наивный байесовский классификатор.

Теорема Байеса используется для вычисления условной вероятности класса для некоторого объекта. Если $X$ &mdash; объект, а $Y \in \{1, ..., N\}$ &mdash; соответствующий ему класс, причем $X$ и $Y(X)$ имеют случайную природу, то условная вероятность класса выражается по формуле
$$\mathsf{P}(Y = k\:|\:X=x) = \frac{\mathsf{P}(Y=k)\:p(x\:|\:Y=k)}{\sum\limits_{k=1}^N\mathsf{P}(Y=k) \:p(x\:|\:Y=k)},$$
где $p(x\:|\:Y=k)$ &mdash; условная плотность $x$ в классе $k$.

Все параметры условных распределений, а также вероятности $\mathsf{P}(Y=k)$ оцениваются по обучающей выборке, в этом заключается *обучение* данных классификаторов. Оценку класса можно получить, взяв тот класс, который соответствует наибольшему значению оценки условной вероятности класса. 

В библиотеке `sklearn` имеется реализации наивного байесовского классификатора `MultinomialNB`.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

Рассмотрим <a target="_blank" href="https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset"> датасет</a> состоящий из сообщений, которые либо являются спамом, либо нет.

Считаем данные:

In [2]:
df = pd.read_csv("spam.csv")
df.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."



Здесь `ham` &mdash; означает, что сообщение не является спамом и`spam` &mdash; сообщение является спамом.

Отделим данные и целевую переменную:

In [3]:
X=df['sms']
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: sms, Length: 5572, dtype: object

In [4]:
y=df['label']
y

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: label, Length: 5572, dtype: object

Посчитаем количество сообщений каждого класса

In [5]:
y.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

Видим, что в данных есть сильный дисбаланс классов.

Разделим данные на обучающую и тестовую часть

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=True)

Данные в таком виде нельзя передавать модели. Их надо привести к численному виду. 

Целевую переменную привести к численному виду можно так:

In [7]:
y_count_train=(y_train== 'spam').astype(int)

Для преобразования текстовых сообщений воспользуемся `CountVectorizer`, работающему по принципу мешка слов (*bag of words*). Он имеет следующие гиперпараметры (т.е. те, которые задаются пользователем):

* `max_df` &mdash; максимальная доля сообщений, в которых может встречатся слово из словаря;
* `min_df` &mdash; минимальная доля сообщений, в которых может встречатся слово из словаря;
* `max_features` &mdash; максимальное возможное количество выбранных слов, они выбираются среди наиболее частых;
* `stop_words` &mdash;слова, которые не будут добавлены в словарь.

Построим векторное представление для наших сообщений. Для этого нужно объявить объект класса `CountVectorizer`. Далее применяем к обучающим данным функцию `fit_transform`, которая последовательно выполняет следующие функции:
* `fit` &mdash; обучение модели, в данном случае подсчет частот слов и определение словаря;
* `transform` &mdash; по существующему словарю преобразует сообщения в векторы.

In [8]:
vectorizer = CountVectorizer(min_df=0.01, max_df=0.05)

In [9]:
X_count_train = vectorizer.fit_transform(X_train)

Напечатаем весь мешок слов и их количество.

In [10]:
print(len(vectorizer.get_feature_names()))
print(vectorizer.get_feature_names())

174
['150p', 'about', 'after', 'again', 'all', 'already', 'also', 'always', 'am', 'amp', 'an', 'any', 'anything', 'as', 'ask', 'babe', 'back', 'been', 'before', 'by', 'cant', 'care', 'cash', 'claim', 'co', 'com', 'come', 'contact', 'cos', 'customer', 'da', 'day', 'dear', 'did', 'doing', 'don', 'dont', 'even', 'every', 'feel', 'find', 'first', 'free', 'give', 'go', 'going', 'gonna', 'good', 'got', 'great', 'gt', 'guaranteed', 'gud', 'had', 'happy', 'has', 'he', 'help', 'her', 'here', 'hey', 'hi', 'him', 'his', 'home', 'hope', 'im', 'its', 'keep', 'know', 'last', 'late', 'later', 'leave', 'let', 'life', 'like', 'll', 'lol', 'lor', 'love', 'lt', 'make', 'meet', 'message', 'min', 'miss', 'mobile', 'more', 'morning', 'msg', 'much', 'need', 'new', 'next', 'night', 'number', 'off', 'oh', 'one', 'only', 'other', 'our', 'out', 'over', 'per', 'phone', 'pick', 'place', 'please', 'pls', 'prize', 're', 'really', 'reply', 'right', 'said', 'say', 'see', 'send', 'sent', 'service', 'she', 'should', 'sm

Для наглядности, создадим таблицу, у которой названия колонок - это слова из мешка слов, а по сторкам стоят текстовые сообщения, слова в которых заменены на числа, которые представляют собой частоту встречи слова в данном сообщении. Нули на месте тех слов, которых нет в сообщении. 

In [11]:
pd.DataFrame(X_count_train.A, columns=vectorizer.get_feature_names())

,150p,about,after,again,all,already,also,always,am,amp,...,which,who,why,win,won,work,would,www,yeah,yes
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4452,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4453,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4454,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Результатом работы `CountVectorizer` является матрица, её нужно превратить в `numpy array`:

In [12]:
X_count_train=X_count_train.toarray()

Выполним теперь аналогичные преобразования с тестовыми данными:

In [13]:
y_count_test=(y_test=='spam').astype(int)

In [14]:
X_count_test = vectorizer.transform(X_test)

Создадим и обучим модель байесовского классификатора:

In [15]:
mult_nb = MultinomialNB()
mult_nb.fit(X_count_train, y_count_train)

MultinomialNB()

Делаем предсказание:

In [16]:
y_pred = mult_nb.predict(X_count_test)

Оценим качество модели. Используем метрику "точность" (accuracy), которая определяется как доля верно классифицированных объектов. 

In [17]:
accuracy_score(y_count_test, y_pred)

0.9721973094170404

Точность оказалась высокой, но только на неё полагаться нельзя.

Оценим точность при помощи ROC - кривой.

Для этого выведем значения вероятностей относиться к тому или иному классу.

In [18]:
y_pred_proba = mult_nb.predict_proba(X_count_test)

Возьмём только вероятности для положительного класса:

In [19]:
probs = y_pred_proba[:, 1]

Построим ROC-кривую:

In [ ]:
from sklearn.metrics import roc_curve, auc
from matplotlib import pyplot as plt

In [ ]:
fpr, tpr, treshold = roc_curve(y_count_test, probs)
roc_auc = auc(fpr, tpr)
# строим график
plt.plot(fpr, tpr, color='darkorange', label='ROC кривая (area = %0.6f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Пример ROC-кривой')
plt.legend(loc="lower right")
plt.show()

In [ ]:
Чем больше площадь под кривой (AUC), тем модель лучше осуществляет классификацию.